# Imputation
How do we handle rows with missing data? We have some options:
- Drop them
- Impute them (fill them in with some value)
    - Mean
    - Median
    - Misc

This will look like:

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('./data/4_housing_competition/train.csv', index_col='Id')
X_test_full = pd.read_csv('./data/4_housing_competition/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice

X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data`
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# ----------- DROPPING COLUMNS -----------
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

# -------- MEAN IMPUTATION -----------
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64
MAE (Drop columns with missing values):
17837.82570776256
MAE (Imputation):
18062.894611872147


From this we can see that imputation actually performs worse than dropping the columns. This is likely because the columns we dropped were not very informative to begin with. In a real scenario, you would want to try and keep as much data as possible, so imputation is often the better choice. However, it's always good to try both and see which works better for your specific dataset.

Lets see if we can get smarter with our imputation:

In [9]:
frontage_by_subclass = X_train.groupby('MSSubClass')['LotFrontage'].median()

# Do thoughtful imputation, we can use the median LotFrontage for each MSSubClass (MSSubClass is a 'house style' to fill in missing values
X_train['LotFrontage'] = X_train.groupby('MSSubClass')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)

# Apply to validation set using training data medians
for subclass in X_valid['MSSubClass'].unique():
    mask = X_valid['MSSubClass'] == subclass
    if subclass in frontage_by_subclass:
        fill_value = frontage_by_subclass[subclass]
    else:
        # If subclass not in training data, use overall training median
        fill_value = X_train['LotFrontage'].median()

    X_valid.loc[mask, 'LotFrontage'] = X_valid.loc[mask, 'LotFrontage'].fillna(fill_value)


imputer = SimpleImputer()
final_X_train = pd.DataFrame(imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(imputer.transform(X_valid))

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17915.223036529682


And this performs better..

So the general idea is try first:
- Drop columns with missing values
- Mean/Median imputation
- More thoughtful imputation

And then compare. YMMV based on your dataset.